#### OBJETIVO NOTEBOOK
- Recolectar y consolidar los datos históricos del Income Statement, Balance Sheet y Cash Flow de las empresas del S&P 500 con un histórico de 10 años.

#### INSTRUCTIONS
- No ejecutar, solo se ha ejecutado una vez para obtener la base de datos de las 500 empresas del SP5500 con todos los años de histórico. La ejecución duró unas 2 horas.
- El resultado de este codigo es el fichero csv guardado en la carpeta DATA_HIST: sp500_financials_and_prices_141224_finalToReview.csv

# LIBRERIAS

In [ ]:
# datos estructurados
import pandas as pd

# retrieve financial data
!pip install alpha_vantage # Install the entire alpha_vantage package
from alpha_vantage.fundamentaldata import FundamentalData

from alpha_vantage.timeseries import TimeSeries

# others
import time

# API

In [ ]:
# api_key = 'C38X3VW71CWFXGSV' #laiafarres9@gmail.com

# api_key = 'EKB5JIV16QLY4VJJ' #lfarres@europastry.com

api_key = 'S1K0DQLP1XKN14SY' #premium

# EXTRACCIÓN DE DATOS

CREAR OBJETOS NECESARIOS

In [ ]:
# crear un objeto FundamentalData
fd= FundamentalData(api_key, output_format='pandas')

# crear un objeto TimeSeries
ts = TimeSeries(api_key, output_format='pandas')

DEFINICIÓN SCOPE TICKERS

In [ ]:
# URL for Wikipedia page containing the S&P 500 companies
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [ ]:
# la función real_html de la libreria pandas permite leer todas las tablas HTML de la pagina web
# el resultado es una lista de DataFrames, seleccionamos la primera tabla, la columna que nos interesa y lo convertimos a lista
table = pd.read_html(wiki_url)[0]
tickers = table['Symbol'].tolist()
len(tickers)

503

CODIGO PARA OBTENER LOS DATOS

***NO EJECUTAR YA QUE TARDA CASI UNA HORA***

In [ ]:
# crear un dataframe vacio para almacenar todos los income statements
all_financials = pd.DataFrame()

# bucle para obtener los estados financieros de cada ticker
for ticker in tickers:
    try:
        # Obtener el Income Statement de la empresa
        income_statement, meta_data = fd.get_income_statement_annual(ticker)

        # Obtener el Balance Sheet de la empresa
        balance_sheet, meta_data_balance = fd.get_balance_sheet_annual(ticker)

        # Obtener el Cash Flow de la empresa
        cash_flow, meta_data_cash_flow = fd.get_cash_flow_annual(ticker)

        # Agregar una columna con el ticker para identificar los datos
        income_statement['ticker_income'] = ticker
        balance_sheet['ticker_balance'] = ticker
        cash_flow['ticker_cashflow'] = ticker


        # filtra solo las columnas necesarias de cada dataframe
        income_statement_filtered = income_statement[['fiscalDateEnding','totalRevenue', 'depreciation', 'depreciationAndAmortization','ebitda', 'netIncome', 'ticker_income']]
        balance_sheet_filtered = balance_sheet[['totalAssets', 'totalCurrentAssets', 'totalCurrentLiabilities','fiscalDateEnding','cashAndCashEquivalentsAtCarryingValue','inventory', 'currentNetReceivables', 'totalLiabilities', 'currentAccountsPayable', 'totalShareholderEquity', 'commonStockSharesOutstanding', 'shortLongTermDebtTotal', 'ticker_balance']]
        cash_flow_filtered = cash_flow[['fiscalDateEnding','capitalExpenditures', 'cashflowFromInvestment', 'dividendPayout','cashflowFromFinancing','operatingCashflow', 'ticker_cashflow']]

        # Realizar el merge de los DataFrames usando la columna 'date'
        financial_data = income_statement_filtered.merge(balance_sheet_filtered, on=['fiscalDateEnding'], how='outer')
        financial_data = financial_data.merge(cash_flow_filtered, on=['fiscalDateEnding'], how='outer')


        # Convertir 'date' a datetime y filtrar los datos de 2013 en adelante
        financial_data['fiscalDateEnding'] = pd.to_datetime(financial_data['fiscalDateEnding'])
        financial_data = financial_data[financial_data['fiscalDateEnding'].dt.year >= 2013]

        # Obtener precios históricos de la acción
        historical_prices, meta_data_prices = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        historical_prices['completeDate'] = pd.to_datetime(historical_prices.index)
        historical_prices['year'] = historical_prices['completeDate'].dt.year

        # Filtrar para obtener el último precio de cierre ajustado del año
        historical_prices['close'] = historical_prices['5. adjusted close']
        year_end_prices = historical_prices.sort_values(by='completeDate').groupby('year').tail(1)[['close']].reset_index()
        year_end_prices.rename(columns={'close': 'Year-End Price'}, inplace=True)

        # Agregar el año a los datos financieros
        financial_data['year'] = financial_data['fiscalDateEnding'].dt.year

        year_end_prices['year'] = year_end_prices['date'].dt.year

        # Combinar precios de cierre anuales con los datos financieros
        combined_data = pd.merge(financial_data, year_end_prices, left_on=['year'], right_on=['year'], how='left')

        # Agregar los datos combinados al DataFrame global
        all_financials = pd.concat([all_financials, combined_data])

        # Mostrar el progreso
        print(f'Obtención de datos de: {ticker}')

        # Esperar un poco entre las solicitudes para no superar el límite de la API
        time.sleep(5)  # Ajusta el tiempo si es necesario para cumplir con las restricciones de la API

    except Exception as e:
        # En caso de error, imprimir el error y continuar con el siguiente ticker
        print(f'Error al obtener datos de {ticker}: {e}')

# Guardar todos los datos combinados en un archivo CSV
all_financials.to_csv('sp500_financials_and_prices_141224_finalToReview.csv', index=False)

print("Datos financieros y precios de cierre anuales guardados en sp500_financials_and_prices_141224_finalToReview.csv")

Obtención de datos de: MMM
Obtención de datos de: AOS
Obtención de datos de: ABT
Obtención de datos de: ABBV
Obtención de datos de: ACN
Obtención de datos de: ADBE
Obtención de datos de: AMD
Obtención de datos de: AES
Obtención de datos de: AFL
Obtención de datos de: A
Obtención de datos de: APD
Obtención de datos de: ABNB
Obtención de datos de: AKAM
Obtención de datos de: ALB
Obtención de datos de: ARE
Obtención de datos de: ALGN
Obtención de datos de: ALLE
Obtención de datos de: LNT
Obtención de datos de: ALL
Obtención de datos de: GOOGL
Obtención de datos de: GOOG
Obtención de datos de: MO
Obtención de datos de: AMZN
Obtención de datos de: AMCR
Obtención de datos de: AMTM
Obtención de datos de: AEE
Obtención de datos de: AEP
Obtención de datos de: AXP
Obtención de datos de: AIG
Obtención de datos de: AMT
Obtención de datos de: AWK
Obtención de datos de: AMP
Obtención de datos de: AME
Obtención de datos de: AMGN
Obtención de datos de: APH
Obtención de datos de: ADI
Obtención de datos